In [1]:
# Reference:
# https://www.kaggle.com/demetrypascal/fork-of-2heads-looper-super-puper-plate/notebook

kernel_mode = True

# Preparations

Let’s load the packages and provide some constants for our script:

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from tensorflow.keras import layers, regularizers, Sequential, Model, backend, callbacks, optimizers, metrics, losses
import tensorflow as tf
import sys
import os
import random
import json
sys.path.append('../input/iterative-stratification')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import pickle
from pickle import dump, load
import glob

import warnings
warnings.filterwarnings('ignore')

In [3]:
PATH = "../input/lish-moa" if kernel_mode else "/workspace/Kaggle/MoA"
model_output_folder = "../input/2heads-looper-super-puper-markpeng" if kernel_mode \
    else f"{PATH}/2heads-looper-super-puper"
os.makedirs(model_output_folder, exist_ok=True)

# SEEDS = [23]
SEEDS = [23, 228, 1488, 1998, 2208, 2077, 404]
KFOLDS = 10

batch_size = 256
# batch_size = 128

label_smoothing_alpha = 0.0005

P_MIN = label_smoothing_alpha
P_MAX = 1 - P_MIN

In [4]:
# Import train data, drop sig_id, cp_type
train_features = pd.read_csv(f'{PATH}/train_features.csv')

non_ctl_idx = train_features.loc[
    train_features['cp_type'] != 'ctl_vehicle'].index.to_list()

# Drop training data with ctl vehicle
tr = train_features.iloc[non_ctl_idx, :].reset_index(drop=True)

test_features = pd.read_csv(f'{PATH}/test_features.csv')
te = test_features.copy()

In [5]:
train_targets_scored = pd.read_csv(f'{PATH}/train_targets_scored.csv')
Y = train_targets_scored.drop('sig_id', axis=1)
Y = Y.iloc[non_ctl_idx, :].copy().reset_index(drop=True).values

train_targets_nonscored = pd.read_csv(f'{PATH}/train_targets_nonscored.csv')
Y0 = train_targets_nonscored.drop('sig_id', axis=1)
Y0 = Y0.iloc[non_ctl_idx, :].copy().reset_index(drop=True).values

sub = pd.read_csv(f'{PATH}/sample_submission.csv')
sub.iloc[:, 1:] = 0

# Features from t.test

Here I am getting most important predictors

In [6]:
# Import predictors from public kernel
json_file_path = '../input/t-test-pca-rfe-logistic-regression/main_predictors.json' if kernel_mode \
    else "/workspace/Kaggle/MoA/t-test-pca-rfe-logistic-regression/main_predictors.json"

with open(json_file_path, 'r') as j:
    predictors = json.loads(j.read())
    predictors = predictors['start_predictors']

In [7]:
second_Xtrain = tr[predictors].copy().values

second_Xtest = te[predictors].copy().values
second_Xtrain.shape

(21948, 447)

# Keras model

I got idea of **label smoothing** from this notebook: https://www.kaggle.com/kailex/moa-transfer-recipe-with-smoothing

In [8]:
def logloss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, P_MIN, P_MAX)
    return -backend.mean(y_true * backend.log(y_pred) +
                         (1 - y_true) * backend.log(1 - y_pred))

# Inference

In [9]:
numeric_features = [c for c in train_features.columns if c != "sig_id"]
gene_experssion_features = [c for c in numeric_features if c.startswith("g-")]
cell_viability_features = [c for c in numeric_features if c.startswith("c-")]
len(gene_experssion_features), len(cell_viability_features)

(772, 100)

In [10]:
tr = tr.drop(['sig_id', 'cp_type', 'cp_time', 'cp_dose'], axis=1)
te = test_features.drop(['sig_id', 'cp_type', 'cp_time', 'cp_dose'], axis=1)

In [11]:
def preprocessor_1(test, seed, scaler=None, pca_gs=None, pca_cs=None):
    n_gs = 2
    n_cs = 100

    # g-mean, c-mean
    test_g_mean = test[gene_experssion_features].mean(axis=1)

    test_c_mean = test[cell_viability_features].mean(axis=1)

    test_columns = test.columns.tolist()

    test = np.concatenate(
        (test, test_g_mean[:, np.newaxis], test_c_mean[:, np.newaxis]), axis=1)

    # Standard Scaler for Numerical Values
    test = pd.DataFrame(data=scaler.transform(test),
                        columns=test_columns + ["g_mean", "c_mean"])
    test_pca_gs = pca_gs.transform(test[gene_experssion_features].values)

    test_pca_cs = pca_cs.transform(test[cell_viability_features].values)

    # Append Features
    test = np.concatenate((test, test_pca_gs, test_pca_cs), axis=1)

    return test


def preprocessor_2(test, scaler=None):
    # Standard Scaler for Numerical Values
    test = scaler.transform(test)

    return test, scaler


def save_pickle(obj, model_output_folder, name):
    dump(obj, open(f"{model_output_folder}/{name}.pkl", 'wb'),
         pickle.HIGHEST_PROTOCOL)


def load_pickle(model_output_folder, name):
    return load(open(f"{model_output_folder}/{name}.pkl", 'rb'))


def mean_logloss(y_pred, y_true):
    logloss = (1 - y_true) * np.log(1 - y_pred +
                                    1e-15) + y_true * np.log(y_pred + 1e-15)
    return np.mean(-logloss)

In [12]:
tr.shape, te.shape

((21948, 872), (3982, 872))

In [13]:
oof_predictions = np.zeros((tr.shape[0], Y.shape[1]))

y_pred = np.zeros((te.shape[0], 206))
for s in SEEDS:

    random.seed(s)
    np.random.seed(s)
    tf.random.set_seed(s)

    k = 0
    kf = KFold(n_splits=KFOLDS, shuffle=True, random_state=s)
    for train_index, valid_index in kf.split(tr):
        file_name = f"seed{s}_fold{k}"

        print(f"Inferencing on seed{s} fold{k} ......")

        scaler_1 = load_pickle(model_output_folder, f"{file_name}_scaler_1")
        pca_gs = load_pickle(model_output_folder, f"{file_name}_pca_gs")
        pca_cs = load_pickle(model_output_folder, f"{file_name}_pca_cs")
        X_test_1 = preprocessor_1(te, s, scaler_1, pca_gs, pca_cs)

        scaler_2 = load_pickle(model_output_folder, f"{file_name}_scaler_2")
        X_test_2, scaler_2 = preprocessor_2(second_Xtest, scaler_2)

        y_valid_1 = Y[valid_index, :]
        y_valid_2 = Y0[valid_index, :]

        n_features = X_test_1.shape[1]
        n_features_2 = X_test_2.shape[1]

        # Model Definition #

        input1_ = layers.Input(shape=(n_features, ))
        input2_ = layers.Input(shape=(n_features_2, ))

        output1 = Sequential([
            layers.BatchNormalization(),
            layers.Dropout(0.2),
            layers.Dense(512, activation="elu"),
            layers.BatchNormalization(),
            layers.Dense(256, activation="elu")
        ])(input1_)

        answer1 = Sequential([
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            layers.Dense(512, "relu")
        ])(layers.Concatenate()([output1, input2_]))

        answer2 = Sequential([
            layers.BatchNormalization(),
            layers.Dense(512, "elu"),
            layers.BatchNormalization(),
            layers.Dense(256, "relu")
        ])(layers.Concatenate()([output1, input2_, answer1]))

        answer3 = Sequential(
            [layers.BatchNormalization(),
             layers.Dense(256,
                          "elu")])(layers.Concatenate()([answer1, answer2]))

        answer3_ = Sequential([
            layers.BatchNormalization(),
            layers.Dense(256, "relu")
        ])(layers.Concatenate()([answer1, answer2, answer3]))

        answer4 = Sequential([
            layers.BatchNormalization(),
            layers.Dense(
                256,
                kernel_initializer=tf.keras.initializers.lecun_normal(seed=s),
                activation='selu',
                name='last_frozen'),
            layers.BatchNormalization(),
            layers.Dense(
                206,
                kernel_initializer=tf.keras.initializers.lecun_normal(seed=s),
                activation='selu')
        ])(layers.Concatenate()([output1, answer2, answer3, answer3_]))

        # Scored Training #

        answer5 = Sequential(
            [layers.BatchNormalization(),
             layers.Dense(Y.shape[1], "sigmoid")])(answer4)

        m_nn = tf.keras.Model([input1_, input2_], answer5)

        m_nn.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                     loss=losses.BinaryCrossentropy(
                         label_smoothing=label_smoothing_alpha),
                     metrics=logloss)

        # Load final model
        m_nn = tf.keras.models.load_model(
            f'{model_output_folder}/{file_name}_final.h5',
            custom_objects={'logloss': logloss})

        # Generate Submission Prediction #
        fold_submit_preds = m_nn.predict([X_test_1, X_test_2],
                                         batch_size=batch_size)
        y_pred += fold_submit_preds / (KFOLDS * len(SEEDS))
        print(fold_submit_preds[:5, :])

        k += 1

        print('\n')

Inferencing on seed23 fold0 ......
[[0.0005265  0.0017706  0.0020488  ... 0.00290867 0.00182123 0.00510768]
 [0.00015905 0.00209163 0.00137384 ... 0.00110859 0.00460724 0.00375134]
 [0.00256567 0.00134712 0.00241133 ... 0.00190495 0.00318621 0.0049495 ]
 [0.00127552 0.00133196 0.0033446  ... 0.00214282 0.00210862 0.0036477 ]
 [0.00223211 0.00365584 0.0014596  ... 0.00209029 0.00063282 0.00100148]]


Inferencing on seed23 fold1 ......
[[0.00128206 0.00124316 0.00141662 ... 0.00227012 0.00287709 0.00234204]
 [0.00284056 0.00226917 0.00166566 ... 0.00072309 0.00497622 0.00267637]
 [0.00027215 0.00024898 0.0009827  ... 0.00397324 0.00308748 0.00301272]
 [0.00238099 0.00156095 0.00305227 ... 0.00257449 0.00449409 0.00113841]
 [0.00139909 0.00114515 0.00286832 ... 0.00282795 0.00098731 0.00104017]]


Inferencing on seed23 fold2 ......
[[8.2563626e-04 8.9468720e-04 9.1752047e-03 ... 4.9941838e-03
  1.3189072e-03 8.0722827e-04]
 [3.2181102e-05 9.8359611e-05 2.4431478e-04 ... 1.7016534e-04
  3.

In [14]:
oof_predictions = glob.glob(f'{model_output_folder}/oof_*.npy')[0]
oof_predictions = np.load(oof_predictions)

oof_loss = mean_logloss(oof_predictions, Y)
print(f"OOF Validation Loss: {oof_loss:.6f}")

OOF Validation Loss: 0.015887


# Submission

In [15]:
sub.iloc[:, 1:] = y_pred
# sub.iloc[:, 1:] = np.clip(y_pred, P_MIN, P_MAX)

In [16]:
sub

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.001182,0.001249,0.002176,0.016794,0.021351,0.005127,0.003655,0.006553,0.000792,...,0.001546,0.001464,0.003633,0.002065,0.000737,0.000988,0.001988,0.002644,0.005610,0.002263
1,id_001897cda,0.000677,0.001781,0.001665,0.002388,0.001148,0.001630,0.003422,0.008910,0.018963,...,0.001444,0.001928,0.002623,0.000643,0.007348,0.001381,0.002662,0.001112,0.008268,0.003162
2,id_002429b5b,0.001062,0.000849,0.003117,0.022861,0.039256,0.003947,0.005764,0.004266,0.001077,...,0.002278,0.001178,0.004928,0.004340,0.003499,0.001134,0.002674,0.005609,0.003419,0.004531
3,id_00276f245,0.001257,0.001154,0.002511,0.015236,0.009135,0.004539,0.002399,0.003539,0.001107,...,0.001180,0.001669,0.002880,0.024105,0.012148,0.001325,0.002366,0.002154,0.002512,0.003590
4,id_0027f1083,0.001711,0.001554,0.002333,0.015025,0.020108,0.003516,0.005872,0.001790,0.000768,...,0.001421,0.000980,0.004322,0.002725,0.001270,0.001217,0.002211,0.002088,0.000807,0.001626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3977,id_ff7004b87,0.001066,0.001283,0.001477,0.003523,0.004267,0.001985,0.001042,0.004224,0.001457,...,0.001044,0.004398,0.002404,0.201066,0.007359,0.001479,0.003968,0.001418,0.001712,0.001671
3978,id_ff925dd0d,0.004358,0.003495,0.001169,0.006227,0.025123,0.004838,0.004657,0.003374,0.001053,...,0.000952,0.001147,0.002733,0.003051,0.001349,0.001496,0.002205,0.001654,0.000932,0.001598
3979,id_ffb710450,0.001778,0.001188,0.001277,0.009477,0.037607,0.005917,0.003525,0.003462,0.000531,...,0.000835,0.000882,0.002074,0.002892,0.001424,0.001009,0.000911,0.001655,0.000997,0.001521
3980,id_ffbb869f2,0.001704,0.001510,0.001364,0.021085,0.030001,0.005809,0.005613,0.002576,0.000658,...,0.000664,0.000490,0.003075,0.001804,0.001273,0.000842,0.001137,0.001759,0.000784,0.002255


In [17]:
# Set ctl_vehicle to 0
sub.iloc[test_features['cp_type'] == 'ctl_vehicle', 1:] = 0

# Save Submission
# sub.to_csv('submission_2heads-looper-super-puper.csv', index=False)
sub.to_csv('submission.csv', index=False)

In [18]:
sub

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.001182,0.001249,0.002176,0.016794,0.021351,0.005127,0.003655,0.006553,0.000792,...,0.001546,0.001464,0.003633,0.002065,0.000737,0.000988,0.001988,0.002644,0.005610,0.002263
1,id_001897cda,0.000677,0.001781,0.001665,0.002388,0.001148,0.001630,0.003422,0.008910,0.018963,...,0.001444,0.001928,0.002623,0.000643,0.007348,0.001381,0.002662,0.001112,0.008268,0.003162
2,id_002429b5b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,id_00276f245,0.001257,0.001154,0.002511,0.015236,0.009135,0.004539,0.002399,0.003539,0.001107,...,0.001180,0.001669,0.002880,0.024105,0.012148,0.001325,0.002366,0.002154,0.002512,0.003590
4,id_0027f1083,0.001711,0.001554,0.002333,0.015025,0.020108,0.003516,0.005872,0.001790,0.000768,...,0.001421,0.000980,0.004322,0.002725,0.001270,0.001217,0.002211,0.002088,0.000807,0.001626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3977,id_ff7004b87,0.001066,0.001283,0.001477,0.003523,0.004267,0.001985,0.001042,0.004224,0.001457,...,0.001044,0.004398,0.002404,0.201066,0.007359,0.001479,0.003968,0.001418,0.001712,0.001671
3978,id_ff925dd0d,0.004358,0.003495,0.001169,0.006227,0.025123,0.004838,0.004657,0.003374,0.001053,...,0.000952,0.001147,0.002733,0.003051,0.001349,0.001496,0.002205,0.001654,0.000932,0.001598
3979,id_ffb710450,0.001778,0.001188,0.001277,0.009477,0.037607,0.005917,0.003525,0.003462,0.000531,...,0.000835,0.000882,0.002074,0.002892,0.001424,0.001009,0.000911,0.001655,0.000997,0.001521
3980,id_ffbb869f2,0.001704,0.001510,0.001364,0.021085,0.030001,0.005809,0.005613,0.002576,0.000658,...,0.000664,0.000490,0.003075,0.001804,0.001273,0.000842,0.001137,0.001759,0.000784,0.002255
